# Clustering Crypto

In [3]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [9]:
# Use the following endpoint to fetch json data
import json
from pprint import pprint

url = "https://min-api.cryptocompare.com/data/all/coinlist"
res = requests.get(url)
res
data = res.json()
json_data = json.dumps(data)

In [34]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
df = pd.DataFrame(data["Data"])
df = df.T
c_df = df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]
c_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
433,433 Token,N/A,False,N/A,NaN,NaN


In [35]:
df.keys()
# json file keys do not contain "TotalCoinSupply" column as in .csv file below. Further analysis will be using data from .csv file.

Index(['Algorithm', 'AssetLaunchDate', 'AssetTokenStatus', 'BlockNumber',
       'BlockReward', 'BlockTime', 'BuiltOn', 'CoinName', 'ContentCreatedOn',
       'DecimalPoints', 'Description', 'Difficulty', 'FullName', 'Id',
       'ImageUrl', 'IsTrading', 'IsUsedInDefi', 'MaxSupply', 'MktCapPenalty',
       'Name', 'NetHashesPerSecond', 'PlatformType', 'ProofType', 'Rating',
       'SmartContractAddress', 'SortOrder', 'Sponsored', 'Symbol', 'Taxonomy',
       'TotalCoinsMined', 'Url'],
      dtype='object')

In [53]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [54]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df.set_index(crypto_df.columns[0], inplace=True)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [66]:
# Keep only cryptocurrencies that are trading
"""
for row in crypto_df['IsTrading']:
    if crypto_df["IsTrading"] == False:
        print(row)   
"""

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [21]:
# Keep only cryptocurrencies with a working algorithm


In [22]:
# Remove the "IsTrading" column


In [23]:
# Remove rows with at least 1 null value


In [24]:
# Remove rows with cryptocurrencies having no coins mined


In [25]:
# Drop rows where there are 'N/A' text values


In [26]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df


In [27]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm


In [28]:
# Create dummy variables for text features


In [29]:
# Standardize data


### Reducing Dimensions Using PCA

In [30]:
# Use PCA to reduce dimensions to 3 principal components


In [31]:
# Create a DataFrame with the principal components data


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [32]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values


# Create the Elbow Curve using hvPlot


Running K-Means with `k=<your best value for k here>`

In [33]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [34]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [35]:
# Table with tradable cryptos


In [36]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [37]:
# Scale data to create the scatter plot


In [38]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
